In [5]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
jet = mcm.get_cmap('jet')
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
%matplotlib inline 

#filename="Nc_Files/Jianfei_WACCMX_files/waccmx_Fe_Fep_Feb.nc"
#filename="Nc_Files/Jianfei_WACCMX_files/waccmx_Na_Nap_Feb.nc"
filename="Nc_Files/Jianfei_WACCMX_files/waccmx_Mg_Mgp_Feb.nc"

filename2="Nc_Files/Jianfei_WACCMX_files/waccmx_T_Z3_e_Feb.nc"
#filename3="Nc_Files/Jianfei_WACCMX_files/waccmx_winds_Feb.nc"

M = filename[-10:]
Metal=M[0:2]
#print(Metal)

ds = xr.open_dataset(filename)
ds2 = xr.open_dataset(filename2)

#ds2
#ds

In [6]:
time = ds.variables['time']
times = xr.cftime_range(start="0001-02-01", periods=672, freq="1H", calendar="noleap")
times2 = ds["time"].dt.strftime("%Y%m%d %H:%M")
times3 = ds["time"].dt.strftime("%Y%m%d")

#times
#times2
#times3

In [7]:
lon = ds.variables['lon']
lat = ds.variables['lat']
lev = ds.variables['lev']

#M = ds.variables[str(Metal)]
#Mp = ds.variables[str(Metal)+'p']

dst = ds.transpose("lev", ...)
Mt = dst.variables[str(Metal)]
Mpt = dst.variables[str(Metal)+'p']


In [8]:
ds2t = ds2.transpose("lev", ...)
temp = ds2t.variables['T']
elect = ds2t.variables['e']

geopH = ds2t.variables['Z3'] / 1000 #m-> km
Zavg = geopH.mean(('time','lat', 'lon'))

In [9]:
Mtdens = ( Mt * 1e-6 * 100 * lev ) / (1.380503e-23 * temp)
Mptdens = ( Mpt * 1e-6 * 100 * lev ) / (1.380503e-23 * temp)

In [10]:
#Mtdens2 = Mtdens / 1000 # convert to 10^3 cm-3
#Mptdens2 = Mptdens / 1000 # convert to 10^3 cm-3

In [11]:
edens = ( elect * 1e-6 * 100 * lev ) / (1.380503e-23 * temp)

In [ ]:
#Trying to calc height of each grid box (cm) in order to then multiply by density (cm-3) to calculate column density (cm-2)

#dst lat: 96 lon: 144 lev: 126 ilev: 127 time: 672 nbnd: 2
#geopH lev: 126 time: 672 lat: 96 lon: 144

#for i in lev:
#    for j in time:
#        for k in lat:
#            for l in lon:
#                geopH_int = geopH[i+1,j,k,l] - geopH[i,j,k,l]


intind_min = 0
intind_max = 125
intinds = np.arange(intind_min,intind_max)
geopH_int = geopH
#geopH_int #lev: 126 time: 672 lat: 96 lon: 144

latinds = np.arange(0,96)
loninds = np.arange(0,144)
timeinds = np.arange(0,672)

for i in intinds:
    for j in timeinds:
        for k in latinds:
            for l in loninds:
                geopH_int[i,j,k,l] = geopH[i,j,k,l] - geopH[i+1,j,k,l]

    

geopH_int[0,0,0,0] #0.00048828
#geopH_int[-1,0,0,0] #2.9051678
#geopH_int[-2,0,0,0] #-17.272299
#geopH_int[-3,0,0,0] #60.200333
#geopH_int #lev: 126 time: 672 lat: 96 lon: 144

In [ ]:
##### levii=np.linspace(0,126,127)

#create an xarray object with dims lev:125, time:672, lat:96, lon:144
alt_i = np.linspace(0,126,127*672*96*144).reshape((127*672*96*144))
dataset = xr.dataset(
    {
        'alt_i':(['levi', 'time', 'lat', 'lon'], alt_i)
    },
    coords=={
        'levi':xr.dataarray(
            levii,
            name='levi',
            dims=['levi'],
        ),
        'time':xr.dataarray(
            time,
            name='time',
            dims=['levi'],
        ),
        'lat':xr.dataarray(
            lat,
            name='levi',
            dims=['levi'],
            attrs={'long_name':'latitude','units':'degrees_north'}
        ),
        'lon':xr.dataarray(
            lon,
            name='levi',
            dims=['levi'],
            attrs={'long_name':'longitude','units':'degrees_east'}
        )
    }
)


dataset

#print(np.linspace(0,126,127))
#alt_i
#geopH
#
#For i in range(geopH.values):
#    print(coor)

In [ ]:

Mt.sel(lev=10^-3, method='nearest')